### Text classification using CNN
__Задание 1.__ Берем отызывы за лето (из архива с материалами или предыдущего занятия). Учим conv сеть для классификации - выбить auc выше 0.95

Собраны твиты 2-ух тональностей, необходимо произвести классификацию на 2-а класса.

In [24]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping 

In [25]:
import pandas as pd

df_train = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test.csv")
df_val = pd.read_csv("data/val.csv")

In [85]:
max_words = 8000
max_len = 100

# Training
epochs = 20
batch_size = 512
print_batch_n = 100

In [86]:
df_train.head()

,id,text,class
0,0,alisachachkaич уезжаааааааать ❤ тожена хотеть ...,0
1,1,rt galyginvadim ребята девчата кино любовь зав...,1
2,2,rt artemklyushin ктоненавидеть пробка ретвит rt,0
3,3,rt epupybobv хотеться котлета покиевск запретн...,1
4,4,karinekurganova yessboss босапоп есбосан боять...,1


In [87]:
df_train['class'].value_counts()

1    92063
0    89404
Name: class, dtype: int64

#### Предобработка

In [29]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [84]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [30]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [16]:
train_corpus = " ".join(df_train["text"])
train_corpus = train_corpus.lower()

In [17]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sych_\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [18]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [88]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [89]:
tokens_filtered_top[:10]

['rt',
 'd',
 'хотеть',
 'знать',
 'ян',
 'мочь',
 'новый',
 'любить',
 'завтра',
 'мой']

In [90]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [91]:
import numpy as np

def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [92]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["text"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["text"]], dtype=np.int32)
x_val = np.asarray([text_to_sequence(text, max_len) for text in df_val["text"]], dtype=np.int32)

In [93]:
x_train.shape

(181467, 100)

In [94]:
x_test.shape

(22684, 100)

In [95]:
x_val.shape

(22683, 100)

#### Keras model

In [105]:
num_classes = 2
y_train = keras.utils.to_categorical(df_train["class"], num_classes)
y_val = keras.utils.to_categorical(df_val["class"], num_classes)

In [106]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [108]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=[keras.metrics.AUC()])

In [109]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
319/319 [==============================] - 59s 185ms/step - loss: 0.5538 - auc: 0.7830 - val_loss: 0.5200 - val_auc: 0.8148
Epoch 2/20
319/319 [==============================] - 58s 182ms/step - loss: 0.4808 - auc: 0.8469 - val_loss: 0.5234 - val_auc: 0.8136


In [110]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test roc auc:', score[1])

45/45 [==============================] - 2s 45ms/step - loss: 0.5216 - auc: 0.8149: 0s - loss: 0.5245 - auc: 0


Test loss: 0.5215526819229126
Test roc auc: 0.8148554563522339


In [40]:
results = model.predict(x_test, batch_size=batch_size, verbose=1)

45/45 [==============================] - 1s 19ms/step


In [41]:
results

array([[0.4743357 , 0.52566427],
       [0.47991362, 0.5200864 ],
       [0.71497685, 0.28502318],
       ...,
       [0.37919787, 0.6208021 ],
       [0.71976084, 0.28023916],
       [0.47737026, 0.52262974]], dtype=float32)